In [1]:
#  Description : Data processing codes GRAPE instrument data 

# Date    : 2024-05-28 16:45:06
# Author  : Karla Onate Melecio <kgom.astro@gmail.com>
# Version : 3.10.12

import sys
import os
import struct
import pathlib
import logging
import time
import warnings

import re

# os.environ['NUMEXPR_MAX_THREADS'] = '12'
# os.environ['NUMEXPR_NUM_THREADS'] = '8'

# logging.basicConfig(level=logging.INFO,
#                     format='%(asctime)s - %(levelname)s - %(message)s',
#                     handlers=[
#                         logging.FileHandler("nb_log.txt"),
#                         logging.StreamHandler()
#                     ])


import pandas as pd
import numpy as np
import uproot


sys.path.append("functions/")
import data_level_functions
from data_level_functions import *

# User Settings

In [ ]:
# Adjust as needed (okay to skip raw data processing if input is .root)
raw_file_directory = '../data/raw/'  # expects .root or .ldat files to be in directory
raw_file_format = '.root'
output_file_format = '.feather' # can specify .pkl.gzip, .csv, or .feather - .root output not currently supported; but can be implented


L0_file_directory = '../data/L0/'

# this step produces the look up tables for coincident events
L2_output_directory = "../data/L2/"
L2_lookup_directory = "../data/ancillary/"
coincidence_window = 30000 # this is the time window for coincident events 

# Raw

**This step puts the raw file in a readable format and is primarily intended for .ldat files, but .root is supported. The returned file only contained three columns: 'time', 'channelID', 'energy'. You can skip this step if your input file is a .root file**


The raw data is first pre-processed with the convert_raw_data_format function which process the raw .ldat file from the Petsys system into a pandas DataFrame containing the unprocessed raw data. convert_raw_data_format utilizes a conversion function called convert_binary_to_dictionary. The convert_binary_to_dictionary function processes a binary file containing petsys "singles" data (all triggers) by reading it in chunks of 16 bytes, each representing a timestamp (ps time tag relative to system start time), energy value (digitized measure of charge), and 'channelID' (detector identification number). It then unpacks these values and stores them in lists. Once all data is read, it constructs a dictionary with keys 'time', 'channelID', and 'energy', where each key corresponds to the respective list of values. The convert_raw_data_format function, offers the flexibility to specify the output format, including Feather, compressed CSV, or compressed Pickle files. This step converts binary data into a structured format and saves it in various file formats specified by the user for further analysis or storage.

The process_raw_data_directory function iterates through the .ldat files in a specified directory. For each file, it extracts the run ID, invokes the convert_raw_data function to process the binary file into a structured DataFrame, and saves it in the desired output format. 


In [ ]:
process_raw_data_directory(directory=raw_file_directory , output_format=output_file_format, input_format=raw_file_format)

# L0

The following functions form a pipeline for preprocessing raw data to the L0 data level.

1. **convert_raw_adc_to_adc(raw_adc_df)**: This function takes a DataFrame containing raw ADC data and converts it to ADC values using a specific formula. It drops the 'energy' column and returns a DataFrame with 'adc' values.

2. **remove_adc_glitches(adc_df, new_glitch_min=None, new_glitch_max=None)**: This function removes glitches from ADC data. It filters the ADC values based on predefined conditions and optionally allows specifying new glitch intervals. It returns a DataFrame with glitches removed.

3. **L0_data_preprocess(input_data, input_data_format, output_format, run_id=None)**: This is the main function that orchestrates the preprocessing pipeline. It takes input data in various formats (pickle, CSV, feather, or DataFrame), converts it to a DataFrame, applies ADC conversion, removes glitches, and optionally saves the processed data in L0 format. If run_id is provided, it saves the processed data; otherwise, it returns the cleaned DataFrame.


The convert_raw_adc_to_adc function accepts a DataFrame containing raw ADC data and computes the corresponding ASIC-corrected ADC values based on predefined coefficients. It applies a mathematical formula to each energy value in the DataFrame, resulting in a new column named 'adc' representing the corrected ADC values. The remove_adc_glitches function then filters out known glitches from the ADC data, removing values falling within specified intervals. Both functions provide information on the number of triggers before and after filtering. Finally, the L0_data_preprocess function preprocesses raw data to the L0 data level, converting ADC channels to ADC values if necessary and removing glitches. It saves the processed data in the specified output format, appending the run ID to the output file name if provided.

In [ ]:
process_L0_data_from_raw(raw_directory=raw_file_directory, L0_directory=L0_file_directory, input_format=raw_file_format, output_format=output_file_format)

# L1

### L1 data is currently under revision, this is data with ancillary data included, but this step is currently being done in the processing using lookup tables. Refer to section labeled "Under Revision" at the bottom of this document.

# L2

The process_L0_file function extracts L0 data file and returns the processed dataframes along with the run ID. The identify_events function processes events from the time-sorted data based on a specified time window, identifies singles events, and generates an event lookup DataFrame. The preprocess_L2_data function preprocesses L0 data files to extract singles events, saves the processed singles data, and event lookup information. It iterates over all L0 data files in a directory, processes each file, and saves the results to specified output directories.

In [ ]:
# L2

preprocess_L2_data(L0_file_directory, L2_output_directory, L2_lookup_directory, window=coincidence_window)


# Under Revision

## L1

In [ ]:
# def process_L1_data_from_L0(L0_directory, L1_directory):
#     """
#         Processes L0 data files from the L0 directory, adds ancillary information (columnID, scintillatorType, PosZ, PosX, PosY),
#         and saves the processed data into the L1 directory in both Feather and Pickle formats.

#         Parameters:
#         L0_directory (str): Path to the directory containing L0 data files.
#         L1_directory (str): Path to the directory where L1 data files will be saved.

#         Returns:
#         None
#     """
#     # gets new datafiles with picosecond resolution
#     data = pathlib.Path(L0_directory)

#     # creates a list of the paths to the binary files in the data folder
#     files = list(data.glob("*.feather"))

#     for i in files:
#         s = str.split(str(i), sep='L0\\')[1]
#         rid = re.search('(.*).feather', s).group(1)
#         rid = re.search('L0_(.*)', rid).group(1)

#         print(rid)
#         data_0 = pd.read_feather(i)
#         data_1 = add_ancillaries_to_data(data_0, identifying_data='../data/ancillary/identifying_data.csv', drop_cols=['biasID', 'pinID'])
#         logging.info('L1 data created')

#         data_1.to_feather(L1_directory + 'L1_' +rid+'.feather')
#         logging.info('L1 data feathered')

#         data_1.to_pickle(L1_directory + 'L1_' +rid + '.pkl.gzip', compression={'method': 'gzip', 'compresslevel': 1, 'mtime': 1})
#         logging.info('L1 data pickled')
#     return

# process_L1_data_from_L0("../data/L0/", "../data/L1/")

## L3

### L3A1

In [ ]:
# def process_l2a1_file(file_path, adc_summary=None, nbins=601, min_adc=0, max_adc=300):
#     """
#     Process an L2A1 file to produce a DataFrame containing binned ADC spectra for each channelID.

#     Parameters:
#     ----------
#     file_path : str or Path
#         Path to the L2A1 feather file.
#     nbins : int
#         Number of bins for the ADC histogram.
#     min_adc : int
#         Minimum ADC value for histogram range.
#     max_adc : int
#         Maximum ADC value for histogram range.

#     Returns:
#     -------
#     DataFrame
#         A DataFrame containing binned ADC spectra for each channelID.
#     """
#     # Read the feather file
#     data = pd.read_feather(file_path)
    
#     # Filter out calibration source detectors
#     data = data[(data.channelID != 81) & (data.channelID != 83)]
#     data.reset_index(drop=True, inplace=True)

#     # Extract run ID from the filename
#     filename = file_path.stem
#     run_id = re.search('L2A1_(.*)', filename).group(1)
    
#     # Initialize an empty DataFrame to store results
#     results = pd.DataFrame()

#     # Get unique channel IDs
#     channel_ids = np.unique(data.channelID)
    
#     # Process each channel ID
#     for channel_id in channel_ids:
#         # Filter data for the current channel ID
#         channel_data = data[data['channelID'] == channel_id]['adc']

#         # if len(adc_summary)>0:
#         #     min_adc = adc_summary[adc_summary['detID']==channel_id].min_adc.values
#         #     max_adc = adc_summary[adc_summary['detID']==channel_id].max_adc.values
#         #     nbins = (max_adc - min_adc)+1
#         #     print(channel_id, nbins)
        
#         # Compute histogram
#         hist, bin_edges = np.histogram(channel_data.values.round(1), bins=nbins, range=[min_adc, max_adc])
        
#         # Create a DataFrame for this channel
#         df = pd.DataFrame({
#             'channelID': channel_id,
#             'bin_edges': bin_edges[:-1].round(1),  # Use left edges of bins
#             'counts': hist
#         })
        
#         # Append to the results DataFrame
#         results = pd.concat([results, df], ignore_index=True)
    
#     return results, run_id

# def process_l2_directory(directory, adc_summary=None, nbins=601, min_adc=0, max_adc=300):
#     """
#     Process all L2A1 files in the given directory to produce a list of DataFrames containing binned ADC spectra for each channelID.

#     Parameters:
#     ----------
#     directory : str or Path
#         Path to the directory containing the L2A1 files.
#     nbins : int
#         Number of bins for the ADC histogram.
#     min_adc : int
#         Minimum ADC value for histogram range.
#     max_adc : int
#         Maximum ADC value for histogram range.

#     Returns:
#     -------
#     tuple
#         A tuple containing a list of DataFrames and a list of run IDs.
#     """
#     data_dir = pathlib.Path(directory)
#     files = list(data_dir.glob("*.feather"))

#     all_results = []
#     run_id_list = []

#     for file_path in tqdm(files, desc="Processing files"):
#         file_results, run_id = process_l2a1_file(file_path, adc_summary, nbins, min_adc, max_adc)
#         all_results.append(file_results)
#         run_id_list.append(run_id)

#     return all_results, run_id_list

# # Example usage
# directory = "../data/L2/"
# adc_spectra_list, run_id_list = process_l2_directory(directory, nbins=2002, min_adc=0, max_adc=1000)


# # Print the first DataFrame and corresponding run_id as an example
# print("First DataFrame:")
# print(adc_spectra_list[0])
# print("Corresponding run_id:", run_id_list[0])